# Deploy Endpoint

This notebook deploys an endpoint from a ModelPackage.

## Import Important Imports

In [ ]:
import sys

sys.path.append("../deploy_model")

import aws_utils as au
import boto3

import sagemaker

## Constants

In [ ]:
AWS_ACCOUNT_ID = "proAdmin"
boto_session = boto3.Session(profile_name=AWS_ACCOUNT_ID)

AWS_ACCOUNT_ID = boto_session.client("sts").get_caller_identity()["Account"]
print(f"Using AWS account {AWS_ACCOUNT_ID}")

sm_role = f"arn:aws:iam::{AWS_ACCOUNT_ID}:role/AmazonSageMaker-ExecutionRole-Marketplace"
sm_b3_session = au.assume_role(boto_session, role_arn=sm_role)

sm_session = sagemaker.Session(boto_session=sm_b3_session)

INFO:numexpr.utils:Note: detected 224 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 224 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
/home/jay/.conda/envs/smfusionp312/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/jay/.config/sagemaker/config.yaml
Using AWS account 423667443377


## Deploy Endpoint

In [4]:
modelpackage_arn = "arn:aws:sagemaker:us-east-1:423667443377:model-package/fused-sgt-llama-8B"

model = sagemaker.ModelPackage(
    role=sm_role,
    model_package_arn=modelpackage_arn,
    sagemaker_session=sm_session,
)

In [8]:
# 8B on g5.4xlarge ~ 10min
model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.4xlarge",
    endpoint_name="fused-sgt-llama-8B-g54x",
    sagemaker_session=sm_session,
)
model.endpoint_name

INFO:sagemaker:Creating model with name: fused-sgt-llama-8B-2025-04-29-20-00-17-308
INFO:sagemaker:Creating endpoint-config with name fused-sgt-llama-8B-g54x
INFO:sagemaker:Creating endpoint with name fused-sgt-llama-8B-g54x


----------------!

'fused-sgt-llama-8B-g54x'